In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AAWW', 'AMCR', 'AVT', 'BCI', 'BHF', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC',
       'COUP', 'ESAB', 'FCFS', 'GEN', 'HUBB', 'IBKR', 'JJC', 'LHCG', 'PEAK',
       'PFG', 'SBNY', 'SIVB', 'STOR', 'UA', 'UMPQ', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['AAWW', 'ACM', 'ACN', 'ADP', 'AGCO', 'AMCR', 'AMED', 'AMG', 'AMGN',
       'ANSS',
       ...
       'VAC', 'VICR', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WTS', 'WTW', 'XEL',
       'YUM'],
      dtype='object', length=209)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['AAWW', 'ABNB', 'AIRC', 'ALGM', 'AM', 'ATCO', 'BHF', 'BJ', 'BRBR',
       'BTC-USD', 'CARR', 'CDAY', 'CEG', 'CHK', 'CHX', 'CNXC', 'COUP', 'CRWD',
       'CTVA', 'DDOG', 'DOW', 'DT', 'DTM', 'ESAB', 'ETH-USD', 'ETRN', 'FOX',
       'FOXA', 'FYBR', 'GEHC', 'GFS', 'GO', 'GXO', 'IAA', 'IR', 'JBGS', 'JHG',
       'JJC', 'KD', 'LCID', 'LHCG', 'MP', 'MRNA', 'NARI', 'NVST', 'NVT', 'OGN',
       'OTIS', 'PDD', 'PGNY', 'RFP', 'RIVN', 'SHC', 'SITM', 'SJI', 'STOR',
       'SWAV', 'UMPQ', 'VICI', 'VNT', 'VSCO', 'WH', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

combined symbols with no volume, same volume and no close, inculdes duplicate symbols: 303
unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 276


In [8]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      29
symbols with same volume:   209
symbols with no close:       65

symbols total before drop:                                        1,595
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   276

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,319    1,500
df Close (df_c) after dropped symbols:      1,319    1,500


In [9]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [10]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [11]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-03-24  130.139999  131.979996  128.220001  131.960007  1571600.0   
2023-03-27  133.580002  134.619995  132.729996  133.070007  1216300.0   
2023-03-28  132.539993  133.309998  132.009995  132.630005   942600.0   
2023-03-29  133.949997  134.240005  132.630005  133.850006  1002800.0   
2023-03-30  135.270004  135.539993  133.320007  134.490005  1468200.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-03-24  39.279999  40.185001  38.816002  40.080002  4013700.0  ...   
2023-03-27  40.509998  40.770000  39.573002  40.119999  3897700.0  ...   
2023-03-28  40.250000  41.029999  40.060001  40.529999  3409800.0  ...   
2023-03-29  41.299999  41.400002  40.709999  40.939999  4154800.0  ...   
2023-03-30  41.599998  42.889999  41.380001  41.759998  4799400.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-03-24  17.320000  17.770000  17.080000  17.620001  442300.0  19.480000   
2023-03-27  17.750000  17.879999  16.990000  17.209999  674300.0  20.200001   
2023-03-28  17.200001  17.910000  17.150000  17.389999  350100.0  20.280001   
2023-03-29  17.610001  17.770000  17.299999  17.730000  398500.0  20.639999   
2023-03-30  18.010000  18.469999  17.730000  17.870001  407500.0  21.120001   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-03-24  19.860001  19.290001  19.850000  1212600.0  
2023-03-27  20.420000  19.969999  20.360001  1530500.0  
2023-03-28  20.690001  20.200001  20.379999  1564700.0  
2023-03-29  20.969999  20.549999  20.879999  1320200.0  
2023-03-30  21.129999  20.809999  20.910000   639000.0  

[5 rows x 6595 columns]

In [12]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABB,ABBV,ABC,ABM,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-03-24,131.960007,40.080002,13.72,110.830002,160.250000,33.490002,31.830999,158.020004,156.410004,43.430000,...,29.959999,125.799919,288.730011,75.209999,42.560001,28.250000,28.510000,164.179993,17.620001,19.850000
2023-03-27,133.070007,40.119999,13.94,114.739998,158.279999,34.860001,32.139999,157.539993,156.690002,44.009998,...,30.049999,125.759995,288.859985,75.820000,41.900002,29.330000,28.150000,165.710007,17.209999,20.360001
2023-03-28,132.630005,40.529999,13.98,115.739998,157.649994,35.430000,32.340000,158.250000,158.000000,44.040001,...,30.250000,126.139999,290.149994,75.349998,41.630001,29.950001,28.790001,165.039993,17.389999,20.379999
2023-03-29,133.850006,40.939999,14.35,117.550003,160.770004,35.689999,32.799999,158.000000,158.970001,44.450001,...,30.440001,126.739998,296.119995,75.839996,42.590000,31.040001,28.270000,162.660004,17.730000,20.879999
2023-03-30,134.490005,41.759998,14.47,118.610001,162.360001,35.950001,33.650002,157.919998,159.339996,44.509998,...,31.770000,126.760002,309.109985,75.830002,42.169998,30.299999,28.430000,164.149994,17.870001,20.910000
